This notebook will query xlsx file using duckdb 
- re-process those faculty with missing scholar_id
- dedup and cleanup

In [1]:
from scrap_cs_faculty import *

### read df

In [5]:
file_xlsx = "cs_faculty_top6-2023-09-04.xlsx"
file_xlsx = "data/cs_faculty_top6-2023-09-10.xlsx"

In [6]:
df = pd.read_excel(Path(file_xlsx))

In [7]:
df.columns 

Index(['name', 'affiliation', 'interests', 'num_papers', 'num_coauthors',
       'citedby', 'hindex', 'i10index', 'citedby5y', 'hindex5y', 'i10index5y',
       'phd_univ', 'phd_year', 'url_homepage', 'url', 'url_author',
       'research_area', 'research_concentration', 'research_focus',
       'job_title', 'url_picture', 'img_url', 'phone', 'email', 'cell_phone',
       'office_address', 'department', 'school', 'scholar_id', 'file_author',
       'not_found_date'],
      dtype='object')

In [8]:
df.fillna("", inplace=True)

In [9]:
df

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author,not_found_date
0,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json,
1,Robert Miller,"Professor, Computer Science, MIT",HCI; software engineering; crowdsourcing,1491.0,0.0,35488.0,78.0,324.0,13243.0,49.0,...,https://www.eecs.mit.edu/people/rob-miller/,(617) 324-6028,rcm@mit.edu,,Office: 32-G718,Computer Science,Massachusetts Institute Technology,e-c3R8QAAAAJ,data/GScholar_robert_miller_mit.json,
2,"Kanade, Takeo",Carnegie Mellon University,Computer Vision,1302.0,0.0,147292.0,169.0,636.0,33531.0,74.0,...,,(412) 268-3016,kanade@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,LQ87h3sAAAAJ,data/GScholar_kanade_takeo_cmu.json,
3,"Faloutsos, Christos",CMU,Data Mining; Graph Mining; Databases,1126.0,51.0,112459.0,148.0,589.0,34139.0,87.0,...,,(412) 268-1457,christos@andrew.cmu.edu,,7003 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,nd8lQQIAAAAJ,data/GScholar_faloutsos_christos_cmu.json,
4,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,Shahar Dobzinski,Unknown affiliation,,75.0,0.0,3177.0,30.0,40.0,1229.0,20.0,...,,,@stanford.edu,,,Computer Science,Stanford Univ,teS9lHAAAAAJ,data/GScholar_shahar_dobzinski_stanford.json,
773,Marco Vassena,Utrecht University,Security; Programming Languages,15.0,15.0,194.0,8.0,7.0,180.0,7.0,...,,,vassena@stanford.edu,,Gates 478,Computer Science,Stanford Univ,q7-1wpcAAAAJ,data/GScholar_marco_vassena_stanford.json,
774,Sanjit A. Seshia,"Professor of EECS, University of California, B...",Formal Methods; Design Automation; Cyber-Physi...,436.0,18.0,21771.0,70.0,194.0,13615.0,57.0,...,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-643-6968,sseshia@eecs.berkeley.edu,,"566 Cory Hall,",Computer Science,Univ California Berkeley,SlZavnIAAAAJ,data/GScholar_sanjit_a_seshia_berkeley.json,
775,Stella Yu,"Professor of EECS, University of Michigan",Computer Vision; Machine Learning; Human Vision,144.0,122.0,10531.0,43.0,88.0,7895.0,32.0,...,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,,,"367 Evans Hall,",Computer Science,Univ California Berkeley,uqWkLzMAAAAJ,data/GScholar_stella_yu_berkeley.json,


In [10]:
sql_stmt = f"""
select name,scholar_id,count(*) from df group by name,scholar_id having count(*) > 1
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,name,scholar_id,count_star()
0,Peter Szolovits,1LuGqFQAAAAJ,2
1,Michael Stonebraker,,2
2,Matei Zaharia,I1EvjZsAAAAJ,4


In [11]:
sql_stmt = f"""
select * from df where scholar_id = '910z20QAAAAJ'
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author,not_found_date
0,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json,


In [12]:
sql_stmt = f"""
select distinct school from df
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,school
0,Univ Illinois Urbana-Champaign
1,Massachusetts Institute Technology
2,Carnegie Mellon Univ
3,Stanford Univ
4,Univ California Berkeley
5,


In [13]:
df_b.values

array([['Univ Illinois Urbana-Champaign'],
       ['Massachusetts Institute Technology'],
       ['Carnegie Mellon Univ'],
       ['Stanford Univ'],
       ['Univ California Berkeley'],
       ['']], dtype=object)

In [14]:
alias_map = {
    'Univ Illinois Urbana-Champaign':"UIUC",
    'Massachusetts Institute Technology':"MIT",
    "Carnegie Mellon Univ":"CMU",
    'Stanford Univ':"Stanford",
    'Univ California Berkeley':"Berkeley",
}

### transform df

In [15]:
df["school_alias"] = df["school"].map(alias_map)

In [16]:
def concat_name_school(r):
    # don't use r.name
    name = r["name"] if isinstance(r["name"],str) else str(r["name"])
    school = r["school_alias"] if isinstance(r["school_alias"],str) else str(r["school_alias"])
    return f"{name} {school}"
        
df["name_school"] = df.apply(concat_name_school, axis=1)

### query df

In [17]:
sql_stmt = f"""
select distinct school,school_alias from df
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,school,school_alias
0,Univ Illinois Urbana-Champaign,UIUC
1,Massachusetts Institute Technology,MIT
2,Carnegie Mellon Univ,CMU
3,Stanford Univ,Stanford
4,Univ California Berkeley,Berkeley
5,,NaN


In [18]:
sql_stmt = f"""
select 
distinct name, school_alias as school, scholar_id 
from df
where scholar_id is Null or scholar_id = ''
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,name,school,scholar_id
0,"DeYoung, Henry",CMU,
1,"Eckhardt, Dave",CMU,
2,"Gligor, Virgil",CMU,
3,"Jahanian, Farnam",CMU,
4,"Kohlbrenner, Anne",CMU,
...,...,...,...
113,Anand V Natarajan,MIT,
114,Randy H. Katz,Berkeley,
115,Matus Jan Telgarsky,UIUC,
116,Babak Ayazifar,Berkeley,


In [19]:
sql_stmt = f"""
select distinct
name, school, school_alias, name_school 
from df
where (scholar_id is Null or scholar_id = '') 
-- and name like 'Moritz%'
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,name,school,school_alias,name_school
0,"DeYoung, Henry",Carnegie Mellon Univ,CMU,"DeYoung, Henry CMU"
1,"Eckhardt, Dave",Carnegie Mellon Univ,CMU,"Eckhardt, Dave CMU"
2,"Gligor, Virgil",Carnegie Mellon Univ,CMU,"Gligor, Virgil CMU"
3,"Jahanian, Farnam",Carnegie Mellon Univ,CMU,"Jahanian, Farnam CMU"
4,"Kohlbrenner, Anne",Carnegie Mellon Univ,CMU,"Kohlbrenner, Anne CMU"
...,...,...,...,...
113,Matus Jan Telgarsky,Univ Illinois Urbana-Champaign,UIUC,Matus Jan Telgarsky UIUC
114,Serge Plotkin,Stanford Univ,Stanford,Serge Plotkin Stanford
115,Michael J. Clancy,Univ California Berkeley,Berkeley,Michael J. Clancy Berkeley
116,David A. Patterson,Univ California Berkeley,Berkeley,David A. Patterson Berkeley


In [20]:
df_out = df_b[["name","name_school"]]

In [21]:
df_out

,name,name_school
0,"DeYoung, Henry","DeYoung, Henry CMU"
1,"Eckhardt, Dave","Eckhardt, Dave CMU"
2,"Gligor, Virgil","Gligor, Virgil CMU"
3,"Jahanian, Farnam","Jahanian, Farnam CMU"
4,"Kohlbrenner, Anne","Kohlbrenner, Anne CMU"
...,...,...
113,Matus Jan Telgarsky,Matus Jan Telgarsky UIUC
114,Serge Plotkin,Serge Plotkin Stanford
115,Michael J. Clancy,Michael J. Clancy Berkeley
116,David A. Patterson,David A. Patterson Berkeley


In [22]:
df_out.to_csv("cs-faculty-no_scholar_id-0910.csv", index=False)

### dedup

In [20]:
sql_stmt = f"""
select name,scholar_id,count(*) from df group by name,scholar_id having count(*) > 1
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,name,scholar_id,count_star()
0,Daniela Rus,910z20QAAAAJ,4
1,Muriel Médard,6_oL_9IAAAAJ,4
2,Manolis Kellis,lsYXBx8AAAAJ,4
3,Samuel Madden,a1ngrCIAAAAJ,4
4,Hari Balakrishnan,Qf4bw4UAAAAJ,4
5,Martin Rinard,hxlxVEUAAAAJ,4
6,Wojciech Matusik,wbIMbL8AAAAJ,4
7,Piotr Indyk,oOwNKsAAAAAJ,4
8,Peter Szolovits,1LuGqFQAAAAJ,4
9,Dina Katabi,nst5fHgAAAAJ,4


In [35]:
sql_stmt = f"""
with rk as (
    select 
    dense_rank() OVER (PARTITION BY scholar_id,name order by affiliation) as rank_id
    , *
    from df 

)
select * from rk order by name
"""
df_b = duckdb.sql(sql_stmt).df()
df_b 

,rank_id,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,1,A. Kevin Tang,,,,,,,,,...,,https://www.cs.cornell.edu/sites/default/files...,,,,,"Electrical and Computer Engineering, CS Field ...",,,
1,1,Aaron Sidford,Stanford University,Algorithms; Optimization; and Numerical Analysis,132.0,0.0,6692.0,43.0,85.0,5752.0,...,https://scholar.google.com/citations?view_op=m...,,,sidford@stanford.edu,,Huang 358,Computer Science,Stanford Univ,SWqd2rgAAAAJ,data/GScholar_aaron_sidford_stanford.json
2,1,Aaron Wagner,"Professor, Electrical and Computer Engineering...",,133.0,29.0,2268.0,23.0,56.0,1093.0,...,https://scholar.google.com/citations?view_op=m...,https://www.cs.cornell.edu/sites/default/files...,,,,,Electrical and Computer Engineering; CS Field ...,,jouwbEQAAAAJ,data/GScholar_aaron_wagner_cornell.json
3,1,Abdussalam Alawini,"Teaching Assistant Professor, University of Il...",Database Systems; Data Science; Data Provenanc...,38.0,12.0,178.0,9.0,8.0,168.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 300-8116,alawini@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,lyoVis4AAAAJ,data/GScholar_abdussalam_alawini_uiuc.json
4,1,Abe Davis,Cornell University,Computer Science; Computer Graphics; Computer ...,53.0,19.0,2460.0,16.0,21.0,1744.0,...,https://scholar.google.com/citations?view_op=m...,https://www.cs.cornell.edu/sites/default/files...,,,,,"Computer Science, CS Field Member",,-Gp8ghkAAAAJ,data/GScholar_abe_davis_cornell.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,1,"Zhang, Hui","Carnegie Mellon University, Conviva",computer networks; streaming; multicast; big data,322.0,0.0,38506.0,89.0,182.0,5389.0,...,https://scholar.google.com/citations?view_op=m...,,,huiz1@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,UMll0FcAAAAJ,data/GScholar_zhang_hui_cmu.json
860,1,"Zheng, Wenting",,,,,,,,,...,,,,wzheng2@andrew.cmu.edu,,9015 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,,
861,1,Zhikun Zhang,,,,,,,,,...,,,,zhikun@stanford.edu,,,Computer Science,Stanford Univ,,
862,1,Zhiru Zhang,,,,,,,,,...,,https://www.cs.cornell.edu/sites/default/files...,,,,,"Electrical and Computer Engineering, CS Field ...",,,


In [38]:
df_b[df_b["scholar_id"].isin(["910z20QAAAAJ", "Kv9AbjMAAAAJ"])]

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
170,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
341,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json


In [27]:
sql_stmt = f"""
select * from df
"""
df_raw = duckdb.sql(sql_stmt).df()
df_raw

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json
1,Robert Miller,"Professor, Computer Science, MIT",HCI; software engineering; crowdsourcing,1491.0,0.0,35488.0,78.0,324.0,13243.0,49.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/rob-miller/,(617) 324-6028,rcm@mit.edu,,Office: 32-G718,Computer Science,Massachusetts Institute Technology,e-c3R8QAAAAJ,data/GScholar_robert_miller_mit.json
2,"Kanade, Takeo",Carnegie Mellon University,Computer Vision,1302.0,0.0,147292.0,169.0,636.0,33531.0,74.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-3016,kanade@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,LQ87h3sAAAAJ,data/GScholar_kanade_takeo_cmu.json
3,"Faloutsos, Christos",CMU,Data Mining; Graph Mining; Databases,1126.0,51.0,112459.0,148.0,589.0,34139.0,87.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-1457,christos@andrew.cmu.edu,,7003 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,nd8lQQIAAAAJ,data/GScholar_faloutsos_christos_cmu.json
4,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,Matus Jan Telgarsky,,,,,,,,,,...,,https://ws.engr.illinois.edu/directory/viewpho...,(217) 300-4507,mjt@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,,
860,Luther Tychonievich,,,,,,,,,,...,,https://ws.engr.illinois.edu/directory/viewpho...,+1 217 333 8609,luthert@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,,
861,Tiffani L. Williams,,,,,,,,,,...,,https://ws.engr.illinois.edu/directory/viewpho...,,tiffani@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,,
862,Michael Joseph Woodley,,,,,,,,,,...,,https://ws.engr.illinois.edu/directory/viewpho...,(217) 244-1971,mwoodley@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,,


In [29]:
df_raw[df_raw["scholar_id"].isin(["910z20QAAAAJ", "Kv9AbjMAAAAJ"])]

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json
4,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
5,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
6,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
7,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
